In [1]:
from utils import list_images, read_image, extract_pixel_size
from pathlib import Path
import napari
import pyclesperanto_prototype as cle

cle.select_device("RTX")

<NVIDIA GeForce RTX 4090 Laptop GPU on Platform: NVIDIA CUDA (1 refs)>

In [2]:
# Copy the path where your images are stored, you can use absolute or relative paths to point at other disk locations
directory_path = Path("./raw_data/condition_1")

# Define the channels you want to analyze using the following structure:
# markers = [(channel_name, channel_nr, cellular_location),(..., ..., ...)]
# cellular locations can be "nucleus", "cytoplasm" or "cell" (cell being the sum volume of nucleus and cytoplasm)
# Remember in Python one starts counting from 0, so your first channel will be 0
# i.e. markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]

markers = [("ki67", 0, "nucleus"), ("neun", 1, "cell"), ("calbindin", 2, "cytoplasm")]

# Image size reduction (downsampling) to improve processing times (slicing, not lossless compression)
# Now, in addition to xy, you can downsample across your z-stack
slicing_factor_xy = 4 # Use 2 or 4 for downsampling in xy (None for lossless)
slicing_factor_z = 1 # Use 2 to select 1 out of every 2 z-slices

# Iterate through the .czi and .nd2 files in the raw_data directory
images = list_images(directory_path)

images

['raw_data\\condition_1\\01.nd2']

In [3]:
image = images[0]

img, filename = read_image(image, slicing_factor_xy, slicing_factor_z)

pixel_size_x, pixel_size_y, voxel_size_z = extract_pixel_size(images[0])

# Correct pixel sizes upon downsampling (slicing_factor =! None)
pixel_size_x = pixel_size_x * slicing_factor_xy
pixel_size_y = pixel_size_y * slicing_factor_xy
voxel_size_z = voxel_size_z * slicing_factor_z



Image analyzed: 01
Original Array shape: (3, 31, 2720, 2720)
Compressed Array shape: (3, 31, 680, 680)
Pixel size: 0.065 µm x 0.065 µm
Voxel (Z-step) size: 0.200 µm


In [4]:
img.shape

(3, 31, 680, 680)

In [5]:
def make_isotropic(image, scaling_x_um, scaling_y_um, scaling_z_um):
    """Scale the image with the voxel size used as scaling factor to get an image stack with isotropic voxels"""
    # Set the x and y scaling to 1 to avoid resizing (compressing the input image), this way the rescaling factor is only applied to z
    multiplier = 1 / scaling_x_um

    scaling_x_um = scaling_x_um * multiplier
    scaling_y_um = scaling_y_um * multiplier
    scaling_z_um = scaling_z_um * multiplier

    image_resampled = cle.scale(
        image,
        factor_x=scaling_x_um,
        factor_y=scaling_y_um,
        factor_z=scaling_z_um,
        auto_size=True,
    )

    return image_resampled

In [17]:
nuclei_resampled = make_isotropic(img[0], pixel_size_x, pixel_size_y, voxel_size_z)

# Remove background with a top_hat_filter
#background_subtracted = cle.top_hat_box(nuclei_resampled, radius_x=5, radius_y=5, radius_z=5)

# Apply gaussian blur to prevent the formation of holes upon labeling
post_gaussian = cle.gaussian_blur(nuclei_resampled, sigma_x=2, sigma_y=2, sigma_z=2)

# Voronoi-Otsu labeling
segmented = cle.voronoi_otsu_labeling(post_gaussian, spot_sigma=10, outline_sigma=1)

# Close holes in labels to avoid false emtpy volumes within the nuclei
closed_labels = cle.closing_labels(segmented, radius=5)

In [18]:
viewer = napari.Viewer(ndisplay=2)

viewer.add_image(nuclei_resampled)
viewer.add_labels(closed_labels)

<Labels layer 'closed_labels' at 0x2288922be50>